<a href="https://colab.research.google.com/github/paramzzz/paramzzz/blob/main/REAL-TIME%20TRAFFIC%20MONITORING%20SYSTEM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests

# Replace with your OpenRouteService API key
ORS_API_KEY = '5b3ce3597851110001cf6248ac9bd7689924491b812a74de277c35ca'

def get_geocode(location):
    geocode_url = f"https://api.openrouteservice.org/geocode/search?api_key={ORS_API_KEY}&text={location}"
    response = requests.get(geocode_url)

    if response.status_code != 200:
        raise Exception(f"Error fetching geocode: {response.status_code}")

    data = response.json()
    if 'features' not in data or len(data['features']) == 0:
        raise Exception("No geocoding results found.")

    coords = data['features'][0]['geometry']['coordinates']
    return coords

def get_traffic_data(start, end):
    try:
        start_coords = get_geocode(start)
        end_coords = get_geocode(end)
    except Exception as e:
        return {'error': str(e)}

    routing_url = f"https://api.openrouteservice.org/v2/directions/driving-car"
    headers = {
        'Authorization': ORS_API_KEY,
        'Content-Type': 'application/json'
    }
    body = {
        "coordinates": [start_coords, end_coords]
    }

    response = requests.post(routing_url, json=body, headers=headers)

    if response.status_code != 200:
        return {'error': f"Error fetching route: {response.status_code}"}

    route_data = response.json()

    if 'routes' not in route_data:
        return {'error': 'No routes found'}

    route = route_data['routes'][0]
    summary = route['summary']

    traffic_info = {
        'start_address': start,
        'end_address': end,
        'distance': summary['distance'] / 1000,  # convert to km
        'duration': summary['duration'] / 60,  # convert to minutes
        'polyline': route['geometry']
    }

    return traffic_info

def display_traffic_info(start, end):
    traffic_info = get_traffic_data(start, end)
    if 'error' in traffic_info:
        print("Error:", traffic_info['error'])
    else:
        print("Traffic Information:")
        print(f"Start Address: {traffic_info['start_address']}")
        print(f"End Address: {traffic_info['end_address']}")
        print(f"Distance: {traffic_info['distance']} km")
        print(f"Duration: {traffic_info['duration']} minutes")
        print("Polyline:", traffic_info['polyline'])

# Hardcoded start and end locations
start = "chennai,IND"
end = "bangalore,IND"
display_traffic_info(start, end)



Traffic Information:
Start Address: chennai,IND
End Address: bangalore,IND
Distance: 331.896 km
Duration: 268.61333333333334 minutes
Polyline: {`wnAi{shNt@yCcBYg@Cg@M}DgDeAzAeApAML}AnBoA~AMNiAjAyBvC{@jAgAvAMPa@f@y@`AQ`@mA~A]\o@^UDkC@iADWBWj@WdBEb@i@fFc@tAIbACXAPEZC\KnASdDCn@MhCM`BaAnDMb@a@t@_DhF_@f@Ob@Kd@Y|EIdE@jCJ`BFxD@x@@~@Bp@JbAf@lFRjBDh@^fC\dDH|BA~AWtCEx@BlAFn@\`C|@vFf@nCv@hE?@d@pCf@tAP\LVlAxAXZb@hALrAAhBE~B?jD@|APn@NZjBnBz@nAt@|AnA|CtApEdBhGn@fEj@zDj@jCXbApAbDn@~A|A|Br@xAdBlCj@|@jB|Cf@bAVp@HVLj@Hv@DbB@tAFdA@^VjAd@fAx@vAlBxC`@n@dC`EFJrDpGNXLXXbAHz@DfAChCB|B@lAPdAZ|@p@`Ar@`A\d@hD`EPRxBrBT^JNTr@T`AJx@?fC?lDKvFEpFCjCIrAANUfDC|@@`AJ|@JZzAnDn@`BNb@Lj@Hj@Bn@@j@C`Aw@fIUrA]hAgAlCEHqA~CGVM|@C`@ApAL`NPfD^hGXvERnGHdCRlGLxHDzBRjHJdBRvARdARt@p@vBn@fBdCfH~GrSp@dB^v@PV`@p@DHfDnEzFtHp@dALZV|@VfB`@vCR|AB`@?~@G|@OdAeBdHiAfF_AhEu@nDkBdJKv@QtBCv@AhBPjK?dFObH[|Ie@`JGvBY|P?tBBdAVfDpAhI\xCT`CHfA^dIT~BJf@d@dBfBnF`ApCp@|An@~@j@p@v@r@^XbAh@~@b@jIdDfEfBVNVLh@XzA|@`@ZNLd@j@^z@H`@H`@LbAJvCP~CVnBJr@XnAJ^b@bB@Bh